In [1]:
import torch.nn as nn
import torch.autograd as autograd
import torch
import pandas as pd
from pathlib import Path
import numpy as np
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import random
from sklearn.metrics import roc_auc_score

In [69]:
PATH = Path("./")

In [70]:
df_train = pd.read_csv("training.csv")
df_train.head(3)

,user_id,item_id,context_feature_id
0,0,28366,2
1,0,16109,2
2,0,11500,3


In [46]:
df_test = pd.read_csv("test_kaggle.csv")
df_test = df_test.drop(columns = ["id"])

In [47]:
df_train.context_feature_id.unique()

array([2, 3, 1, 0])

In [48]:
df_item_f = pd.read_csv("item_feature.csv")

In [49]:
df_train = df_train.merge(df_item_f,how="left",on = "item_id")
df_test = df_test.merge(df_item_f,how="left",on = "item_id")

In [50]:
uniq_items = df_train.item_id.unique()
uniq_user = df_train.user_id.unique()

In [ ]:
context_id_mode = df_train.groupby(['user_id']).context_feature_id.agg(lambda x: pd.Series.mode(x)[0]).reset_index()
context_id_mode.head()

In [10]:
dict_user_cnt =dict(df_train.groupby(['user_id']).item_id.count())

In [11]:
max_item_new = list(range(max(uniq_items)))

In [12]:
negative_users = []
for user in dict_user_cnt:
    items = np.random.choice(max_item_new, size =dict_user_cnt[user], replace= False )
    for item in items:
        negative_users.append( (user, item) )

In [13]:
df_neg = pd.DataFrame(negative_users, columns=['user_id', 'item_id'])

In [14]:
df_neg = df_neg.merge(context_id_mode, how= "left", on="user_id")
df_neg = df_neg.merge(df_item_f,how="left",on = "item_id")
df_neg.head(5)

,user_id,item_id,context_feature_id,item_feature_id
0,0,17385,2,187
1,0,16121,2,62
2,0,9438,2,2
3,0,2433,2,142
4,0,24425,2,18


In [15]:
df_train.insert(df_train.shape[1],'rating',1)
df_neg.insert(df_neg.shape[1],'rating',0)

In [16]:
df = pd.concat([df_train, df_neg])

In [17]:
df.groupby('rating').count()

,user_id,item_id,context_feature_id,item_feature_id
rating,,,,
0,970245,970245,970245,970245
1,970245,970245,970245,970245


In [19]:
df.groupby('rating').count()

,user_id,item_id,context_feature_id,item_feature_id
rating,,,,
0,970245,970245,970245,970245
1,970245,970245,970245,970245


In [ ]:
df.to_csv("new_date_with_dup_repeat.csv")

# Read the saved DATA and the test Data

In [24]:
df = pd.read_csv("new_date_with_dup_repeat.csv")

In [25]:
df_test = pd.read_csv("test_kaggle.csv")
df_test = df_test.drop(columns = ["id"])
df_item_f = pd.read_csv("item_feature.csv")
df_test = df_test.merge(df_item_f,how="left",on = "item_id")

# Splitting the Data

In [26]:
msk = np.random.rand(len(df)) < 0.8
train = df[msk].reset_index(drop=True).copy()
valid = df[~msk].reset_index(drop=True).copy()

In [ ]:
def encode_feature(train, valid, test, encode_dict, feature):
    train[feature] = train[feature].apply(lambda x: encode_dict[x])
    valid[feature] = valid[feature].apply(lambda x: encode_dict.get(x, 0))
    test[feature] = test[feature].apply(lambda x: encode_dict.get(x, 0))
    return train, valid, test

In [27]:
# User id
train_user_ids = np.sort(np.unique(train.user_id.values))
userid2idx = {o:i+1 for i,o in enumerate(train_user_ids)}
train, valid, df_test = encode_feature(train, valid, test, userid2idx, "user_id")

# Item_id Encoding
train_item_ids = np.sort(np.unique(train.item_id.values))
itemid2idx = {o:i+1 for i,o in enumerate(train_item_ids)}
train, valid, df_test = encode_feature(train, valid, test, itemid2idx, "item_id")

#Item Feature id encoding
train_item_feature_ids = np.sort(np.unique(train.item_feature_id.values))
featureid2idx = {o:i for i,o in enumerate(train_item_feature_ids)}
train, valid, df_test = encode_feature(train, valid, test, featureid2idx, "item_feature_id")

#Item Context id encoding 
train_context_feature_ids = np.sort(np.unique(train.context_feature_id.values))
contextid2idx = {o:i for i,o in enumerate(train_context_feature_ids)}
train, valid, df_test = encode_feature(train, valid, test, contextid2idx, "context_feature_id")

# Create DataLoader

In [28]:
class matrix_factorization():
    def __init__(self, x1, x2,x3,x4, y):
        self.x1 = torch.LongTensor(x1) 
        self.x2 = torch.LongTensor(x2)
        self.x3 = torch.LongTensor(x3)
        self.x4 = torch.LongTensor(x4)
        self.y = torch.FloatTensor(y)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self,idx):
        return self.x1[idx], self.x2[idx], self.x3[idx], self.x4[idx],self.y[idx]

In [29]:
train_ds = matrix_factorization(train.user_id.values,train.item_id.values,\
                                train.item_feature_id.values,train.context_feature_id.values, train.rating.values)

valid_ds = matrix_factorization(valid.user_id.values,valid.item_id.values,\
                                valid.item_feature_id.values,valid.context_feature_id.values, valid.rating.values)

In [30]:
train_dl = DataLoader(train_ds, batch_size = 15000, shuffle = True)
valid_dl = DataLoader(valid_ds, batch_size = 15000, shuffle = False)

In [42]:
class MF(nn.Module):
    def __init__(self, num_users, num_items,num_feature,num_context, emb_size = 100, emb_extra = 5, layer_size_1 = 60,layer_size_2= 10,frac = 0.2):
        super(MF, self).__init__()
        self.dropout = nn.Dropout(frac)
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        
        #init
        self.user_emb.weight.data.uniform_(0,0.05)
        self.item_emb.weight.data.uniform_(0,0.05)
        self.Linear_1 = nn.Linear(emb_size*2, layer_size_1) 
        self.Linear_2 = nn.Linear(layer_size_1, layer_size_2)
        self.output_ = nn.Linear(layer_size_2, 1)
        
    def forward(self, u,v,f,c):
        U = self.user_emb(u)
        V = self.item_emb(v)
        x = self.Linear_1(torch.cat((U,V),1))
        x = torch.relu(x)
        x = self.Linear_2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.output_(x)
        return x 
        

In [31]:
# here we are not using data loaders because our data fits well in memory
def train_epocs(model,train_dl, valid_dl, epochs, optimizer,scheduler):
    
    for i in range(epochs):
        print(f"epoch no: {i}")
        losses = []
        model.train()
        for users, items, item_feature, context_id, ratings in train_dl:
            msk = np.random.rand(users.shape[0]) < 0.6
            users[~msk] = 0
            y_hat = model(users, items, item_feature, context_id)
            loss = F.binary_cross_entropy_with_logits(y_hat.float(), ratings.unsqueeze(1).float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        scheduler.step()
        
        #print(losses)
        train_loss = np.mean(losses)
        valid_loss,valid_auc = val_metrics(model, valid_dl)
        print("train loss %.3f valid loss %.3f auc-roc %.3f" % (train_loss, valid_loss, valid_auc))

In [32]:
def val_metrics(model, valid_dl):
    model.eval()
    losses = []
    y_hats = []
    ys = []
    for users, items, item_feature, context_id, ratings in valid_dl:
        y_hat = model(users, items, item_feature, context_id)
        loss = F.binary_cross_entropy_with_logits(y_hat.float(), ratings.unsqueeze(1).float())
        y_hats.append(y_hat.detach().numpy())
        ys.append(ratings.numpy())
        losses.append(loss.item())
    ys = np.concatenate(ys)
    y_hats = np.concatenate(y_hats)
    #print(y_hats.shape,ys.shape )
    return np.mean(losses), roc_auc_score(ys, y_hats)

In [33]:
num_users = len(train_user_ids)
num_items = len(train_item_ids)
num_feature = len(df.item_feature_id.unique())
num_context = len(df.context_feature_id.unique())

In [34]:
model = MF(num_users+1, num_items+1,num_feature,num_context, emb_size = 75, emb_extra = 50,
           layer_size_1 = 20,layer_size_2 = 5,frac = 0.15)

NameError: name 'MF' is not defined

In [62]:
from torch.optim.lr_scheduler import StepLR,ExponentialLR
learning_rate = 0.001
wd=0.2
epochs = 4
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate,weight_decay=wd)
#scheduler = StepLR(optimizer, step_size=5, gamma=0.1,verbose=False)
scheduler = ExponentialLR(optimizer, gamma=0.9)

In [63]:
train_epocs(model,train_dl, valid_dl, epochs, optimizer,scheduler)

epoch no: 0
train loss 0.397 valid loss 0.481 auc-roc 0.892
epoch no: 1
train loss 0.384 valid loss 0.478 auc-roc 0.890
epoch no: 2
train loss 0.373 valid loss 0.476 auc-roc 0.890
epoch no: 3


KeyboardInterrupt: 

In [64]:
user_ = torch.LongTensor(df_test.user_id.values) # .cuda()
item_ = torch.LongTensor(df_test.item_id.values) #.cuda()
feature_ = torch.LongTensor(df_test.item_feature_id.values) # .cuda()
context_ = torch.LongTensor(df_test.context_feature_id.values) #.cuda()

y_hat = model(user_, item_, feature_, context_)
prob = pd.Series(torch.sigmoid(y_hat).flatten().detach().numpy()).reset_index().rename(columns={"index":'id',0:"rating"})
prob.to_csv("submission_31",index=False)